In [1]:
import sys
# Specify path to Remo
# Windows
#local_path_to_repo =  'C:/Users/crows/Documents/GitHub/remo-python'
# Mac
local_path_to_repo = '/Users/melodi/Docs/remo-python'
sys.path.insert(0, local_path_to_repo)

In [2]:
%load_ext autoreload
%autoreload 2
import remo


    (\(\ 
    (>':') Remo server is running: {'version': '0.3.5-184-g2d9bc73'}
                


## Get Dataset

In [3]:
remo.list_datasets()

[Dataset 1 - 'Vova's dataset',
 Dataset 2 - 'open images detection',
 Dataset 3 - 'Subset of OID',
 Dataset 4 - 'train',
 Dataset 6 - 'test',
 Dataset 12 - 'oid 100',
 Dataset 13 - 'open images sample data',
 Dataset 19 - 'oid building ',
 Dataset 21 - 'oid building and car',
 Dataset 22 - 'oid building car detection',
 Dataset 25 - 'open images data',
 Dataset 26 - 'oid car',
 Dataset 28 - 'oid car and person',
 Dataset 29 - 'test upload',
 Dataset 30 - 'test upload 2',
 Dataset 31 - 'cats and dogs']

In [3]:
my_dataset = remo.get_dataset(31)
my_dataset.view()

Open http://localhost:8000/datasets/31


![dataset](view_data2.gif)

In [6]:
my_dataset.view_annotation_statistics()

Open http://localhost:8000/annotation-detail/32/intro


![annotation](view_annotation_statistics.gif)

## Train-Test Split

In [4]:
len(my_dataset)

200

In [4]:
val = my_dataset[0:len(my_dataset) // 3]
train = my_dataset[len(my_dataset) // 3:]
val.export_annotation_to_csv('val_cats_dogs.csv')
train.export_annotation_to_csv('train_cats_dogs.csv')

We export our train and validation sets into .csv files in the format of:
  
    file_name,class
	cat.8.jpg,Cat
	cat.9.jpg,Cat
	cat.10.jpg,Cat
    ..


## Prepare for Image Classification

In [5]:
from example.classification import *

### Arrange Train & Validation Data

In [7]:
# TODO: check .csv reader in PyTorch
arrange_folders(classes=['Cat','Dog'], root='/Users/melodi/dogscats/sample/')

Now we can continue to build our classification model with PyTorch.

## PyTorch

In [8]:
train_dataset, train_dataloader = prepare_datasets(root_path='/Users/melodi/dogscats/sample/', phase='train')
val_dataset, val_dataloader = prepare_datasets(root_path='/Users/melodi/dogscats/sample/', phase='val')

In [9]:
model_ft = train_model(train_dataset, val_dataset, train_dataloader, val_dataloader, model_path='model_cat_dog.pth')

Best Val Acc: 0.8636363636363636


Now we will feed our validation dataset into the model and combine the predictions in a dataframe and export as .csv to upload Remo.

In [11]:
df_preds = predict(train_dataset, val_dataloader, model_ft, model_path='/Users/melodi/Docs/remo-python/example/model_cat_dog.pth')

In [12]:
df_preds.to_csv('preds_raw_cats_dogs.csv')

## Visualize Predictions in Remo

Now, we can create a new annotation set called "Predictions" and push our prediction annotations.

In [13]:
my_dataset.create_annotation_set(annotation_task='Image classification', name='Predictions',classes=["Dog","Cat"], path_to_annotation_file='/Users/melodi/Docs/remo-python/example/preds_raw_cats_dogs.csv')

In [14]:
my_dataset.view()

Open http://localhost:8000/datasets/31
